In [ ]:
%pylab inline
%autosave 20
import os
os.environ['KERAS_BACKEND']='theano'
import keras
import sys
sys.path.append('/home/peter')
sys.path.append('/home/peter/code/projects/MultiNEAT')
sys.path.append('/home/ubuntu')
from universal import *
sys.path.append('/home/peter/code/projects')
sys.path.append('/home/peter/code/projects/deepneat')
sys.path.append('/home/ubuntu')
sys.path.append('/home/ubuntu/new/automl')
from aidevutil import *
from keras.utils.vis_utils import plot_model
from tqdm import tqdm_notebook as tqdm
from deepneat import *

In [ ]:
from dask import compute, delayed, persist
from dask.distributed import Client, wait
from dask.distributed import as_completed

In [ ]:
import ipywidgets as widgets

In [ ]:
(x_train, y_train), (x_test, y_test) = get_mnist(size=(28,28), nrows=1000, nrows_test=1000)
dx = x_train.reshape(x_train.shape[0], -1)
dy = y_train.reshape(-1, 1)

dx_test = x_test.reshape(x_test.shape[0], -1)

In [ ]:
dx.shape, dy.shape

In [ ]:
dx.shape, dy.shape

In [ ]:
###################
# Prototype Nodes #
###################
#
# Linear, PCA
#
# LogisticRegression
#
#

In [ ]:
# NEAT Parameters 
params = NEAT.Parameters()

params.PopulationSize = 300
params.DynamicCompatibility = True
params.YoungAgeTreshold = 3
params.SpeciesMaxStagnation = 10
params.OldAgeTreshold = 10
params.MinSpecies = 1
params.MaxSpecies = 12
params.RouletteWheelSelection = False
params.ArchiveEnforcement = False
params.InnovationsForever = True

params.ConstraintTrials = 128

params.MutateAddNeuronProb = 0.15/3
params.MutateAddLinkProb = 0.5/3
params.MutateRemLinkProb = 0.1/3
params.RecurrentProb = 0.0
params.MaxWeight = 1.0

params.MutateWeightsProb = 0.0
params.MutateActivationAProb = 0.0
params.MutateActivationBProb = 0.0
params.MutateNeuronTimeConstantsProb = 0.0
params.MutateNeuronBiasesProb = 0.0

params.MutateGenomeTraitsProb = 0.0
params.MutateNeuronTraitsProb = 0.5
params.MutateLinkTraitsProb = 0.0

params.OverallMutationRate = 0.5
params.CrossoverRate = 0.7
params.MultipointCrossoverRate = 0.4
params.SurvivalRate = 0.2
params.InterspeciesCrossoverRate = 0.005
params.PreferFitterParentRate = 0.5

params.DontUseBiasNeuron = True
params.AllowLoops = False
params.AllowClones = True

params.ExcessCoeff = 1.0
params.DisjointCoeff = 1.0

params.WeightDiffCoeff = 0.0
params.TimeConstantDiffCoeff = 0.0
params.BiasDiffCoeff = 0.0
params.ActivationADiffCoeff = 0.0
params.ActivationBDiffCoeff = 0.0
params.NormalizeGenomeSize = True

params.MinCompatTreshold = 0.0
params.CompatTreshold = 1.25
params.CompatTreshChangeInterval_Evaluations = 1
params.CompatTresholdModifier = 0.002

In [ ]:
# This is the custom trait
class BaseObject:
    def __init__(self):
        pass
        
    def mate(self, other):
        baby = BaseObject()
        return baby
    
    def mutate(self):
        baby = BaseObject()
        return baby
    
    def distance_to(self, other):
        return rnd.uniform(0,1)
    
    def conc_inputs(self, mm, inputs):
        if (mm=='concat') or (len(inputs)==1):
            if len(inputs)==1:
                return inputs
            else:
                return np.concatenate(inputs)
        else:
            if mm=='add':
                return np.sum()
    
    # what every object has to do - transform x, possibily changing the number of columns
    # if necessary, fit to data (note: undeterministic!)
    def tick(self, x, y):
        return np.array(x)

In [ ]:
class genePCA(BaseObject):
    def __init__(self):
        self.ndim = rnd.randint(2, 16)
        self.pca = PCA(n_components=self.ndim)
        
    def __repr__(self):
        return "PCA(%d)" % self.ndim
    
    def mate(self, other):
        if isinstance(other, genePCA):
            baby = genePCA()
            baby.ndim = (self.ndim + other.ndim)//2
            baby.pca = PCA(n_components=baby.ndim)
            return baby
        else:
            return rnd.choice([self, other])
    
    def mutate(self):
        baby = genePCA()
        baby.ndim += rnd.randint(-3, 3)
        baby.ndim = np.clip(baby.ndim, 2, 16)
        baby.pca = PCA(n_components=baby.ndim)
        return baby
    
    def distance_to(self, other):
        if isinstance(other, genePCA):
            return float(np.abs(self.ndim - other.ndim))
        else:
            return 100000.0
    
    def tick(self, x, y):
        xt = self.pca.fit_transform(x)
        return np.array(xt)

In [ ]:
class geneLinear(BaseObject):
    def __init__(self):
        pass
    
    def __repr__(self):
        return "Linear"
    
    def mate(self, other):
        return self
    
    def mutate(self):
        return self
    
    def distance_to(self, other):
        return 0
    
    def tick(self, x, y):
        return np.array(x)

In [ ]:
from sklearn.linear_model import LogisticRegression

class geneLogisticRegression(BaseObject):
    def __init__(self):
        self.clf = LogisticRegression()
        
    def __repr__(self):
        return "LogReg"
    
    def mate(self, other):
        return self
    
    def mutate(self):
        return self
    
    def distance_to(self, other):
        return 0
    
    def tick(self, x, y):
        self.clf.fit(x, y.reshape(-1))
        return self.clf.predict(x)

In [ ]:
derived_list = [genePCA, geneLinear, geneLogisticRegression]
probs = [1.0]*len(derived_list)

In [ ]:
def fails_constraints(genome):
    try:
        ts = genome.GetNeuronTraits()
        o = None
        for t in ts:
            if t[1]=='output': 
                o = t[2]
                break
        if not isinstance(o['node'], geneLogisticRegression):
            return True
        else:
            return False
    except:
        return True
    return True

In [ ]:
params.CustomConstraints = fails_constraints

In [ ]:
# Set traits

In [ ]:
# merge mode
s = ['concat',
    # when dimensions match, these can be done, otherwise it defaults to concat
     'add','mul','avg','min','max',
    ]
p = [1.0] * len(s)
mm = {'details': {'set': s, 'probs': p},
      'importance_coeff': 0.0,
      'mutation_prob': 0.3,
      'type': 'str'}

params.SetNeuronTraitParameters('mm', mm)

In [ ]:
node = {'details': (derived_list, probs),
          'importance_coeff': 0.1,
          'mutation_prob': 0.2,
          'type': 'pyclassset'}

params.SetNeuronTraitParameters('node', node)

In [ ]:
num_inputs = 1
num_outputs = 1

In [ ]:
num_input_dims = dx.shape[1]
num_output_dims = 1

In [ ]:
g = NEAT.Genome(0, num_inputs, 1, num_outputs, 0, NEAT.ActivationFunction.RELU,
                    NEAT.ActivationFunction.RELU, 0, params, 0, 1)

In [ ]:
pop = NEAT.Population(g, params, True, 1.0, 0)
pop.RNG.Seed(int(time.clock()*100))

In [ ]:
dx.shape, dy.shape

In [ ]:
dx[0:3]

In [ ]:
dy[0:3]

In [ ]:
gr = NEAT.Genome2NX(pop.Species[0].Individuals[0])

In [ ]:
gr.nodes(data=1)

In [ ]:
numrows = dx.shape[0]

def activate_graph(gr, inputs, targets, num_outputs=1):
    allnodes = list(nx.dfs_postorder_nodes(gr))[::-1]
    for a in allnodes: gr.node[a]['act'] = None
    
    # separate input from non-input nodes
    allnodes = [x for x in allnodes if x > num_inputs]
    
    # input the data
    for i,inp in zip(range(1, num_inputs+1), inputs): 
        gr.node[i]['act'] = np.array(inp).reshape(numrows, -1)
        
    # pass through the graph
    for an in allnodes:
        #print(gr.node[a], end=' ')
        mm = gr.node[an]['mm']
        
        # collect the inputs to this node
        
        # also sort the incoming edges by id for consistency
        inedg = list(gr.in_edges(an))
        inps = [gr.node[i]['act'] for i,o in inedg]
        
        if (mm == 'concat') or (len(inps)==1) or not all([x.shape[1] == inps[0].shape[1] for x in inps]):  
            if len(inps)>1:
                iii = np.concatenate(inps, axis=1)
            else:
                if isinstance(inps, list):
                    iii = inps[0]
                else:
                    iii = inps
        else:
            iii = inps
            if mm == 'add':
                iii = np.sum(inps, axis=0)
            if mm == 'mul':
                iii = np.prod(inps, axis=0)
            if mm == 'avg':
                iii = np.mean(inps, axis=0)
            if mm == 'min':
                iii = np.min(inps, axis=0)
            if mm == 'max':
                iii = np.max(inps, axis=0)
        
        act = gr.node[an]['node'].tick(iii, targets).reshape(numrows, -1)
        
        # store activation
        gr.node[an]['act'] = act 

    outputs = [gr.node[o]['act'] for o in allnodes[-num_outputs:]]
    return np.array(outputs).reshape(numrows, -1)

In [ ]:
a = activate_graph(gr, [dx], dy)

In [ ]:
a.shape

In [ ]:
#a#gr.nodes(data=1)

In [ ]:
# evaluation function

def evaluate(args):
    idx, gr, dx, dy, ltr, ntr, precomp = args
    ntr = [x for x in ntr if x[1]!='input']
    
    try:
        a = activate_graph(gr, [dx], dy)
        if a.shape[1] > 1: 
            a = np.sum(a, axis=1)
            a = np.round((np.tanh(a) + 1.0)/2.0)
        acc = np.sum(a == dy) / numrows
    except Exception as ex:
        print(ex)
        acc = 0.0
    
    f = acc
    
    return idx, f, None

def decide(x):
    return x 

In [ ]:
if socket.gethostname() == 'laptop':
    cluster='192.168.0.108:8786'
elif socket.gethostname() == 'desktop':
    clsuter='192.168.0.108:8786'
else:
    cluster='172.31.43.76:8786'

In [ ]:
import time
st = time.time()
print('Connecting..')
if cluster:
    client = Client(cluster)
else:
    client = Client()
client.restart(timeout=600)
# push the data to the cluster
print('Scattering data to cluster..')
debugxy=0
if not debugxy:
    fx = client.scatter(dx, broadcast=True, direct=True)
    fy = client.scatter(dy, broadcast=True, direct=True)
else:
    fx, fy = None, None
print(f'done in {time.time()-st} seconds')

In [ ]:
#evaluate((0, NEAT.Genome2NX(pop.Species[0].Individuals[2]), dx, dy))

In [ ]:
parallel = 0
precomp=None
verbose=1
use_local_search=0
display_whole_pop = 0
display_max_species = 8
initeval = 0
evaluations = 10000
display_pop_each = 80000
once = False
x_shape, ys = dx.shape[1], 1
population = params.PopulationSize

penalize_stangation = 1
penalize_stagnation_evals = 2400

max_stagnation = 5000

max_nodeslinks_to_output = 4

evhist = []
best_ever = 0
best_gs = []

In [ ]:
def poplen(pop):
    return sum([len(x.Individuals) for x in pop.Species])

In [ ]:
def prettydict(d, nonl=False):
    i,t,di = d
    if not nonl:
        ks = '%d - %s: ' % (i, t)
    else:
        return '%d - %s' % (i, t)
    s = []
    for k,v in sorted(list(di.items())):
        if isinstance(v, float):
            s.append( '%s %3.3f' % (k[0:1], v) )
        else:
            s.append( '%s %s' % (k[0:1], v) )
    return ks + ', '.join(s)

In [ ]:
def species_display(pop):
    genomes = [x.GetLeader() for x in pop.Species][0:display_max_species]
    f, axes = plt.subplots(1, len(genomes), figsize=(len(genomes) * 4.5, 14))
    print('Species Representatives:')
    if len(genomes)>1:
        for i,(ax, g) in enumerate(zip(axes, genomes)):
            try:
                img = NEAT.viz.Draw(g, size=(300, 400))[:, 0:230]
                ax.imshow(img)
                ax.set_title('%3.6f | %3.2f%%' % (decide(g.GetFitness()),
                                                  (len(pop.Species[i].Individuals)/poplen(pop))*100 ))
                s = '\n'.join([prettydict(x) for x in g.GetNeuronTraits() if x[1] != 'input'][0:8])
                s += '\n=============\n'
                s += '\n'.join([prettydict(x, nonl=True) for x in g.GetLinkTraits(False)][0:8])
                ax.set_xlabel(s)
            except Exception as ex:
                print(ex)
    else:
        try:
            g = pop.GetBestGenome()
            img = NEAT.viz.Draw(g, size=(300, 400))[:, 0:230]
            axes.imshow(img)
            axes.set_title('%3.6f' % (decide(g.GetFitness())))
            s = '\n'.join([prettydict(x) for x in g.GetNeuronTraits() if x[1] != 'input'][0:8])
            s += '\n=============\n'
            s += '\n'.join([prettydict(x, nonl=True) for x in g.GetLinkTraits(False)][0:8])
            axes.set_xlabel(s)
        except Exception as ex:
            print(ex)
    f.tight_layout()
    plt.show()

In [ ]:
once = False
#evhist = []
best_ever = 0
best_gs = []

nfl = 1
if debugxy:
    nfl = 1
g = NEAT.Genome(0, num_inputs, 1, num_outputs, 0, NEAT.ActivationFunction.RELU,
                    NEAT.ActivationFunction.RELU, 0, params, 0, nfl)
pop = NEAT.Population(g, params, True, 1.0, 0)
pop.RNG.Seed(int(time.clock()*100))
#pop.Parameters.AllowClones = False
evhist = []

# rtNEAT code
if 1:
    if initeval:
        if verbose:
            print('============================================================')
            print("Please wait for the initial evaluation to complete.")
        now = time.time()
        genome_list = []
        for s in pop.Species:
            for i in s.Individuals:
                genome_list.append(i)
        sys.stdout.flush()
        # turn them into NX networks
        grlist = [NEAT.Genome2NX(x, with_weights=1) for x in genome_list]
        ltrlist = [x.GetLinkTraits(1) for x in genome_list]
        ntrlist = [x.GetNeuronTraits() for x in genome_list]
        idlist = [x.GetID() for x in genome_list]
        if parallel:
            args = list(zip(idlist,
                            grlist,
                            [fx]*len(grlist),
                            [fy]*len(grlist),
                            ltrlist,
                            ntrlist, 
                            [precomp]*len(grlist)
                            ))
        else:
            args = list(zip(idlist,
                            grlist,
                            [dx]*len(grlist),
                            [dy]*len(grlist),
                            ltrlist,
                            ntrlist,
                            [precomp]*len(grlist)
                            ))

        # evaluate all individuals
        fitnesses = [0] * len(grlist)
        if verbose:
            bar = ProgressBar(initial_value=0, max_value=len(grlist))
        fff = {}
        rss = {}
        if parallel:
            cp = [client.submit(evaluate, x) for x in args]
            for i,ftr in enumerate(as_completed(cp)):
                idx, fitness, rsx  = ftr.result()
                fff[idx] = fitness
                rss[idx] = rsx
                if verbose:
                    bar.update(i)
        else:
            for i in range(len(grlist)):
                idx, fitness, rsx = evaluate(args[i])
                fff[idx] = fitness
                rss[idx] = rsx
                if verbose:
                    bar.update(i)                
        if verbose:
            bar.finish()

        fitness_list = []
        rsx_list = []
        for gnm in genome_list:
            gnm.SetFitness( fff[gnm.GetID()] )
            fitness_list.append( fff[gnm.GetID()] )
            rsx_list.append( rss[gnm.GetID()] )
            if use_local_search:
                for qx,w in zip(gnm.LinkGenes,rss[gnm.GetID()]): qx.Weight = w
            gnm.SetEvaluated()

        bidx = argmax(fitness_list)
        best = fitness_list[bidx]
        #if initeval: evhist.append(decide(best))
        if best > best_ever:
            if verbose:
                print('NEW RECORD!')
                print('Evaluations:', 0,
                      'Fitness:', decide(best),
                      'Species:', len(pop.Species))
            best_gs.append([NEAT.Genome2NX(genome_list[bidx], with_weights=1), 
                            genome_list[bidx].GetLinkTraits(1),
                            genome_list[bidx].GetNeuronTraits(),
                            rsx_list[bidx]
                           ])
            best_bs = best_gs[-3:]
            if verbose:
                print(f'-------- nodes: {genome_list[bidx].NumNeurons()} -------- ')
                print('\n'.join([str(x) for x in genome_list[bidx].GetNeuronTraits() if x[1] != 'input'][0:max_nodeslinks_to_output]))
                print(f'-------- links: {genome_list[bidx].NumLinks()} -------- ')
                print('\n'.join([str(x) for x in genome_list[bidx].GetLinkTraits(1)][0:max_nodeslinks_to_output]))
                print('----------------------- ')
            best_ever = best
            if viz and verbose:
                species_display(pop)

        if verbose:
            print("Evaluation took", '%3.2f' % (time.time() - now), "seconds.")
    else:
        # No initial evaluation, start off with small random fitness
        for s in pop.Species:
            for i in s.Individuals:
                i.SetFitness(np.random.rand()/1000)
                i.SetEvaluated()              

    if parallel:
        genome_list = NEAT.GetGenomeList(pop)
        grlist = [NEAT.Genome2NX(x, with_weights=1) for x in genome_list]
        ltrlist = [x.GetLinkTraits(1) for x in genome_list]
        ntrlist = [x.GetNeuronTraits() for x in genome_list]
        idlist = [x.GetID() for x in genome_list]
        args = list(zip(idlist,
                        grlist,
                        [fx]*len(grlist),
                        [fy]*len(grlist),
                        ltrlist,
                        ntrlist,
                        [precomp]*len(grlist)
                        ))
        cp = [client.submit(evaluate, x) for x in args]

    if verbose:
        print('============================================================')
        print('rtNEAT phase')
        print('============================================================')

        format_custom_text = FormatCustomText('CTresh: %(ctr).6f Species: %(sp)d Last fitness: %(fitness).6f %(vr)s',
            dict(
                sp=0,
                fitness=0.0,
                vr=str([]),
                ctr=0.0,
            ),)

        widgets = ['Evaluated: ', Counter('%(value)d'), ' ', format_custom_text,
                   ' (', Timer(), ') ', RotatingMarker(), FileTransferSpeed(unit='individuals') ]
        bar = ProgressBar(widgets=widgets, initial_value=0, max_value=evaluations)

    if parallel:
        # parallel continious loop
        seq = as_completed(cp)
        i=0
        olds = []
        under_evaluation_now = len(cp)
        for f in tqdm(seq):
            if i >= evaluations:
                [x.cancel() for x in cp]
                break
            if verbose:
                if i > 0: bar.update(i)
            i += 1

            # get result from evaluation
            idx, fitness, rsx = f.result()
            if 1:
                if idx not in olds:
                    # set that individual's fitness
                    pop.AccessGenomeByID(idx).SetFitness(fitness)
                    pop.AccessGenomeByID(idx).SetEvaluated()
                    under_evaluation_now -= 1

                    thegenome = pop.AccessGenomeByID(idx)

                    #  apply the local search back to genome
                    if use_local_search:
                        for qx,w in zip(thegenome.LinkGenes,rsx): qx.Weight = w

                    # get best fitness in population and print it
                    evhist.append(decide(fitness))
                    if fitness > best_ever:
                        if verbose:
                            print('NEW RECORD!')
                            print('Evaluations:', i,
                                  'Fitness:', decide(fitness))
                        best_gs.append([NEAT.Genome2NX(thegenome, with_weights=1),
                                        thegenome.GetLinkTraits(1),
                                        thegenome.GetNeuronTraits(),
                                        rsx
                                       ])
                        if verbose:
                            print(f'-------- nodes: {thegenome.NumNeurons()} -------- ')
                            print('\n'.join([str(x) for x in thegenome.GetNeuronTraits() if x[1] != 'input'][0:max_nodeslinks_to_output]))
                            print(f'-------- links: {thegenome.NumLinks()} -------- ')
                            print('\n'.join([str(x) for x in thegenome.GetLinkTraits(1)][0:max_nodeslinks_to_output]))
                            print('----------------------- ')
                        best_ever = fitness
                        if viz and verbose:
                            species_display(pop)
                    elif (i%display_pop_each)==0: 
                        if viz and verbose:
                            species_display(pop)

                else:
                    if verbose: 
                        print(f'Fitness of genome #{idx} was not set.')

            #except Exception as ex:
            #    print(ex)
            if verbose:
                format_custom_text.update_mapping(fitness=decide(fitness),
                                                  sp=len(pop.Species),
                                                  ctr=pop.Parameters.CompatTreshold)


            # apply exploration pressure to species
            if penalize_stangation:
                maxfit = max([x.GetFitness() for x in NEAT.GetGenomeList(pop)])
                for s in pop.Species:
                    if s.EvalsNoImprovement > penalize_stagnation_evals:
                        bf = max([x.GetFitness() for x in s.Individuals])
                        # make an exception if that species contains the best genome so far
                        if bf < maxfit:
                            for ind in s.Individuals:
                                ind.SetFitness(0.000000001)

            if all([x.EvalsNoImprovement > max_stagnation for x in pop.Species]):
                break

            # create new baby and add to list of tasks
            # only add new babies if the population under evaluation is below N%
            if under_evaluation_now <= (population*(1/3)):
                if not once:
                    once=True
                    if verbose:
                        print(f'rtNEAT reproduction cycle started at evaluation #{i}')

                old = NEAT.Genome()
                baby = pop.Tick(old)

                if old.GetID() != -1:
                    olds.append(old.GetID())

                newf = client.submit(evaluate, (baby.GetID(),
                                                NEAT.Genome2NX(baby, with_weights=1),
                                                fx,
                                                fy,
                                                baby.GetLinkTraits(1),
                                                baby.GetNeuronTraits(),
                                                precomp
                                                ))
                seq.add(newf)
                under_evaluation_now += 1

    else:

        # continuous loop
        for i in tqdm(range(evaluations)):
            if verbose:
                bar.update(i)

            # get the new baby
            old = NEAT.Genome()
            baby = pop.Tick(old)

            # evaluate it
            f = evaluate((baby.GetID(),
                          NEAT.Genome2NX(baby, with_weights=1),
                          dx,
                          dy,
                          baby.GetLinkTraits(1),
                          baby.GetNeuronTraits(),
                          precomp
                          ))

            fitness = f[1]
            rsx = f[2]

            baby.SetFitness(fitness)
            baby.SetEvaluated()

            # apply the local search back to genome
            if use_local_search:
                for qx,w in zip(baby.LinkGenes,rsx): qx.Weight = w

            # get best fitness in population and print it
            #evhist.append(decide(fitness))
            if fitness > best_ever:
                if verbose:
                    print('NEW RECORD!')
                    print('Evaluations:', i,
                          'Fitness:', decide(fitness))
                best_gs.append([NEAT.Genome2NX(baby, with_weights=1),
                               baby.GetLinkTraits(1),
                               baby.GetNeuronTraits(),
                               rsx])
                best_bs = best_gs[-3:]
                if verbose:                    
                    print(f'-------- nodes: {baby.NumNeurons()} -------- ')
                    print('\n'.join([str(x) for x in baby.GetNeuronTraits() if x[1] != 'input'][0:max_nodeslinks_to_output]))
                    print(f'-------- links: {baby.NumLinks()} -------- ')
                    print('\n'.join([str(x) for x in baby.GetLinkTraits(1)][0:max_nodeslinks_to_output]))
                    print('----------------------- ')
                best_ever = fitness
                if viz and verbose:
                    species_display(pop)
            elif (i%display_pop_each)==0: 
                if viz and verbose:
                    species_display(pop)

            if all([x.EvalsNoImprovement > max_stagnation for x in pop.Species]):
                break

            if (i%50000)==0:
                gc.collect()

            #spechist.append([len(pop.Species), pop.Parameters.CompatTreshold])

            if verbose:
                format_custom_text.update_mapping(fitness=decide(f[1]),
                                                  sp=len(pop.Species),
                                                  ctr=pop.Parameters.CompatTreshold)

#except KeyboardInterrupt:
#    pass

#except Exception as ex:
#    print(ex)


In [ ]:
#%debug

In [ ]:
#client.restart(timeout=600)